# Process the data (combine them together)

## Check JH's augmentation data

In [ ]:
import importlib
import functions

JH_aug_mat_path = "/Volumes/Shawn_SSD/PhD/Project/Date/Tmp/JH_Pre_Augmentation/original_augmentation_data/JH_aug_2.mat"
JH_reshape_vol_mat_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Val/Input/FA_17017030_AugVol_2.mat"
JH_reshape_pts_mat_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Val/Output/FA_17017030_AugPts_2.mat"

pixel_space = [0.15, 0.15, 0.15]

importlib.reload(functions)

# JH_aug_volume, JH_aug_pts = functions.load_mat_data(JH_aug_mat_path)
# functions.show_pts(JH_aug_volume, JH_aug_pts, pixel_space)

JH_aug_zoomed_volume, JH_aug_zoomed_pts = functions.load_mat_data(JH_reshape_vol_mat_path, JH_reshape_pts_mat_path)
functions.show_pts(JH_aug_zoomed_volume, JH_aug_zoomed_pts, pixel_space)


In [ ]:
# Try up_sampling method

up_sampling_JH = functions.rescale_3d_volume(JH_aug_zoomed_volume, (512, 512, 90))
functions.show_pts(up_sampling_JH, JH_aug_zoomed_pts, pixel_space)

## Combination

In [ ]:
import numpy as np
import importlib
import functions

def combine_data(x_base_path, y_base_path, store_base_path, data_tag):
    importlib.reload(functions)

    # X_val_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Val/Input"
    # Y_val_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Val/Output"

    x_dataset, y_dataset = functions.load_mat_data_dir(x_base_path, y_base_path)

    x_dataset = np.asarray(x_dataset)
    y_dataset = np.asarray(y_dataset)

    x_dataset_path = store_base_path + "X_" + data_tag + "_data"
    y_dataset_path = store_base_path + "Y_" + data_tag + "_data"

    np.save(x_dataset_path, x_dataset)
    np.save(y_dataset_path, y_dataset)


In [ ]:
X_train_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Train/Input"
Y_train_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Train/Output"
X_val_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Val/Input"
Y_val_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Val/Output"
X_test_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Test/Input"
Y_test_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Test/Output"
store_base_path = "data/"

combine_data(X_train_base_path, Y_train_base_path, store_base_path, "train")
combine_data(X_val_base_path, Y_val_base_path, store_base_path, "val")
combine_data(X_test_base_path, Y_test_base_path, store_base_path, "test")

In [ ]:
import numpy as np
import functions

# Inspect Loaded Data

X_file_path = "data/X_train_data.npy"
Y_file_path = "data/Y_train_data.npy"

X_load = np.load(X_file_path, allow_pickle=True)
Y_load = np.load(Y_file_path, allow_pickle=True)

print(np.shape(X_load), np.shape(Y_load))
print(np.shape(X_load[1, :, :, :]), np.shape(Y_load[1, :, :]))

# functions.show_pts(X_load[1, :, :, :], Y_load[1, :, :], [0.15, 0.15, 0.15])
# functions.show_pts(X_load[2, :, :, :], Y_load[2, :, :], [0.15, 0.15, 0.15])
# functions.show_pts(X_load[3, :, :, :], Y_load[3, :, :], [0.15, 0.15, 0.15])
# functions.show_pts(X_load[4, :, :, :], Y_load[4, :, :], [0.15, 0.15, 0.15])

In [3]:
import Spartan.models as models
import importlib

importlib.reload(models)

# model_s1, model_s2 = models.spine_lateral_radiograph(width=32, height=32, depth=16)
# model_s1.summary()
# model_s2.summary()

model = models.spine_lateral_radiograph(width=32, height=32, depth=16)
model.summary()

Model: "ResModel"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 16,  0           []                               
                                 1)]                                                              
                                                                                                  
 batch_normalization_51 (BatchN  (None, 32, 32, 16,   4          ['input_2[0][0]']                
 ormalization)                  1)                                                                
                                                                                                  
 re_lu_51 (ReLU)                (None, 32, 32, 16,   0           ['batch_normalization_51[0][0]'] 
                                1)                                                         

In [ ]:
from tensorflow import keras

# keras.utils.plot_model(model_s1, "my_first_model.png")

In [4]:
import numpy as np

input = np.arange(2*32*32*16*1).reshape(2,32,32,16,1)
base_cor_xyz = np.arange(2*32*32*16*4*3).reshape(2,32,32,16,4,3)

[output_s1, output_s2] = model([input, base_cor_xyz])

print(np.shape(output_s1))

(2, 4, 3)
